In [ ]:
!pip install nibabel monai timm torchmetrics


In [ ]:
#clear directory
!cd /kaggle/working/
!rm -rf *

In [ ]:
!pip install gdown


In [15]:
import gdown

#https://drive.google.com/file/d/1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ/view?usp=sharing

file_id = "1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ"  # replace with your file ID
url = f"https://drive.google.com/uc?id={file_id}"

output_path = "/kaggle/working/kits23.zip"
gdown.download(url, output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ
From (redirected): https://drive.google.com/uc?id=1tUArfL0QHhd0XkfXX1FnExF8_PbABtDQ&confirm=t&uuid=c8edca36-e1b5-49ce-9a89-5ef3a2a715c8
To: /kaggle/working/kits23.zip
100%|██████████| 1.28G/1.28G [00:15<00:00, 82.4MB/s]


'/kaggle/working/kits23.zip'

In [ ]:
import zipfile

zip_path = "/kaggle/working/kits23.zip"
extract_path = "/kaggle/working/kits23"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
import os, torch, nibabel as nib
import numpy as np
from torch.utils.data import Dataset, DataLoader

class KiTS23_2p5D(Dataset):
    def __init__(self, root_dir, cases, num_slices=3, transform=None):
        self.root_dir = root_dir
        self.cases = cases
        self.num_slices = num_slices
        self.transform = transform
        self.samples = []
        self._prepare_samples()

    def _prepare_samples(self):
        for case in self.cases:
            img_path = os.path.join(self.root_dir, case, "imaging.nii.gz")
            lbl_path = os.path.join(self.root_dir, case, "segmentation.nii.gz")
            img = nib.load(img_path).get_fdata()
            lbl = nib.load(lbl_path).get_fdata()
            img = (img - np.mean(img)) / (np.std(img) + 1e-5)

            for z in range(1, img.shape[2]-1):
                self.samples.append((img[..., z-1:z+2], lbl[..., z]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        stack, mask = self.samples[idx]
        stack = torch.tensor(stack).float().permute(2, 0, 1)  # (3,H,W)
        mask = torch.tensor(mask).long()  # (H,W)
        return stack, mask


In [ ]:
root = "/kaggle/working/kits23/kits23"
cases = sorted(os.listdir(root))[:10]  # use only 50 cases

train_cases = cases[:7]
val_cases = cases[7:]

train_ds = KiTS23_2p5D(root, train_cases)
val_ds   = KiTS23_2p5D(root, val_cases)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=4)


In [ ]:
import torch
print(torch.cuda.is_available())  # should be True
print(torch.cuda.device_count())  # should match Kaggle GPU (1 or 2)
print(torch.cuda.get_device_name(0))


In [ ]:
import torch.nn as nn
from monai.networks.blocks import UnetOutBlock
from timm.models.vision_transformer import vit_base_patch16_224

class TransUNet2p5D(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()
        self.vit = vit_base_patch16_224(pretrained=True)
        self.vit.conv_proj = nn.Conv2d(in_channels, 768, kernel_size=16, stride=16)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(768, 256, 2, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 2, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 2, 2),
            nn.ReLU(),
            nn.Conv2d(64, out_channels, 1)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        y = self.vit.conv_proj(x)
        y = y.flatten(2).transpose(1, 2)
        y = self.vit.blocks(y)
        y = y.transpose(1, 2).reshape(B, 768, H//16, W//16)
        y = self.decoder(y)
        return y


In [ ]:
import torch.nn.functional as F

def dice_loss(pred, target, eps=1e-5):
    pred = torch.softmax(pred, dim=1)
    target_1hot = F.one_hot(target, num_classes=3).permute(0,3,1,2)
    inter = (pred * target_1hot).sum(dim=(2,3))
    denom = (pred + target_1hot).sum(dim=(2,3))
    dice = (2 * inter / (denom + eps)).mean()
    return 1 - dice

model = TransUNet2p5D(in_channels=3, out_channels=3).cuda()
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)


In [ ]:
for epoch in range(10):  # 10–20 epochs enough for baseline
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()
        out = model(x)
        loss = dice_loss(out, y)
        opt.zero_grad(); loss.backward(); opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}: Train DiceLoss {total_loss/len(train_loader):.4f}")


new

In [ ]:
import os
import nibabel as nib
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class KiTS23_2p5D_Resized(Dataset):
    def __init__(self, root_dir, cases, num_slices=3, target_size=(256, 256)):
        """
        root_dir: folder containing case_00000, case_00001, ...
        cases: list of case folder names to use
        num_slices: number of consecutive slices for 2.5D input (usually 3)
        target_size: (H, W) to resize each slice
        """
        self.root_dir = root_dir
        self.cases = cases
        self.num_slices = num_slices
        self.target_size = target_size
        self.samples = []
        self._prepare_samples()

    def _prepare_samples(self):
        for case in self.cases:
            img_path = os.path.join(self.root_dir, case, "imaging.nii.gz")
            lbl_path = os.path.join(self.root_dir, case, "segmentation.nii.gz")

            img = nib.load(img_path).get_fdata()
            lbl = nib.load(lbl_path).get_fdata()

            # Normalize intensity
            img = (img - np.mean(img)) / (np.std(img) + 1e-5)

            # Generate 2.5D slices
            for z in range(1, img.shape[2]-1):
                self.samples.append((img[..., z-1:z+2], lbl[..., z]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        stack, mask = self.samples[idx]

        # Convert to torch tensors
        stack = torch.tensor(stack).float().permute(2, 0, 1)  # (C,H,W)
        mask = torch.tensor(mask).long()                       # (H,W)

        # Resize stack and mask
        stack = F.interpolate(stack.unsqueeze(0), size=self.target_size, mode='bilinear', align_corners=False).squeeze(0)
        mask = F.interpolate(mask.unsqueeze(0).unsqueeze(0).float(), size=self.target_size, mode='nearest').squeeze(0).long()

        return stack, mask


In [ ]:
root = "/kaggle/working/kits23/kits23"
cases = sorted(os.listdir(root))[:10]  # use only 50 cases
train_cases = cases[:7]
val_cases = cases[7:]

train_ds = KiTS23_2p5D_Resized(root, train_cases)
val_ds   = KiTS23_2p5D_Resized(root, val_cases)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=4, num_workers=2)


In [ ]:
import torch.nn as nn
from monai.networks.blocks import UnetOutBlock
from timm.models.vision_transformer import vit_base_patch16_224

class TransUNet2p5D(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()
        self.vit = vit_base_patch16_224(pretrained=True)
        self.vit.conv_proj = nn.Conv2d(in_channels, 768, kernel_size=16, stride=16)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(768, 256, 2, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 2, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 2, 2),
            nn.ReLU(),
            nn.Conv2d(64, out_channels, 1)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        y = self.vit.conv_proj(x)
        y = y.flatten(2).transpose(1, 2)
        y = self.vit.blocks(y)
        y = y.transpose(1, 2).reshape(B, 768, H//16, W//16)
        y = self.decoder(y)
        return y


**Importing model**

In [3]:
!pip install monai 


In [2]:
import monai
print(monai.__version__)


<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-10-19 08:21:20.905848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760862080.929296     117 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760862080.936351     117 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


1.5.1


In [12]:
!pip install --upgrade monai


In [14]:
from monai.networks.nets import UNETR
import torch

model = UNETR(
    in_channels=1,            # input channels (CT=1)
    out_channels=3,           # number of segmentation classes
    img_size=(64, 512, 512),  # D,H,W patch size
    feature_size=16,          # base feature size
    hidden_size=768,          # transformer hidden size
    mlp_dim=3072,             # MLP dimension
    num_heads=12,             # number of attention heads
    norm_name='instance',     # normalization type
    res_block=True            # use residual blocks in encoder
).cuda()


In [ ]:
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class KiTS23Dataset(Dataset):
    def __init__(self, root_dir, cases, patch_size=(64,512,512)):
        self.root_dir = root_dir
        self.cases = cases
        self.patch_size = patch_size

    def __len__(self):
        return len(self.cases)

    def __getitem__(self, idx):
        case = self.cases[idx]
        img = nib.load(f"{self.root_dir}/{case}/imaging.nii.gz").get_fdata()
        lbl = nib.load(f"{self.root_dir}/{case}/segmentation.nii.gz").get_fdata()
        img = (img - np.mean(img)) / (np.std(img)+1e-5)  # normalize

        # simple center crop to patch_size
        d, h, w = self.patch_size
        z = max(0, img.shape[2]//2 - d//2)
        y = max(0, img.shape[0]//2 - h//2)
        x = max(0, img.shape[1]//2 - w//2)
        img_patch = img[y:y+h, x:x+w, z:z+d]
        lbl_patch = lbl[y:y+h, x:x+w, z:z+d]

        img_patch = torch.tensor(img_patch, dtype=torch.float32).unsqueeze(0)  # (1,D,H,W)
        lbl_patch = torch.tensor(lbl_patch, dtype=torch.long)                   # (D,H,W)
        return img_patch, lbl_patch

# Example usage
root = "/kaggle/working/kits23"
cases = sorted(os.listdir(root))[:50]
train_cases = cases[:40]
val_cases = cases[40:]

train_ds = KiTS23Dataset(root, train_cases)
val_ds = KiTS23Dataset(root, val_cases)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=2)
